# Parent-Child Chunk 를 OpenSearch 에 저장

이 노트북은 전체 문서를 Parent, Child 문서로 구분을 하여 오픈 서치 벡터스토어에 저장을 하고, 테스트를 하는 노트북 입니다. 또한 일부 문서로 검증 인덱스를 생성하여 테스트 용으로 만들기 위한 노트북 입니다.

---

## [중요] 사전 실행 노트북
이 노트북은 아래 두개의 셋업 노트북이 먼저 실행이 되어야 합니다.
- (1) Setup 노트북
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup.ipynb 와 같습니다.
    -  [Setup Notebook](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup.ipynb)
- (2) Amazon OpenSearch 설치 노트북    
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb 와 같습니다.
    - [Setup OpenSearch](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb)
    - 위 노트북을 통하여 아래와 같은 정보를 이 노트북에서 사용합니다.
        - opensearch_domain_endpoint
        - opensearch_user_id
        - opensearch_user_password

## [중요] 노트북 생성 결과 
이 노트북을 실행을 하고 나면 Parameter Store 에 두개의 오픈 서치 인덱스 이름이 저장 됩니다. 다음 노트북에서 아래의 변수 이름으로 값을 찾아서 사용하게 됩니다.
- opensearch_index_name
- opensearch_evaluation_index_name

# 1. Bedrock Client 생성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

module_path = "../../.."
add_python_path(module_path)


python path: /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot is added
sys.path:  ['/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/home/ec2-user/anaconda3/envs/python3/lib/python310.zip', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/lib-dynload', '', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages', '/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot']
python path: /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr is added
sys.path:  ['/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/home/ec2-user/anaconda3/envs/python3/lib/python310.zip', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/lib-

In [2]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. Embedding 모델 로딩

## Embedding Model 선택

In [3]:
Use_Titan_Embedding = True
Use_Cohere_English_Embedding = False

## Embedding Model 로딩

In [4]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings

if Use_Titan_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "amazon.titan-embed-text-v1")
    dimension = 1536
elif Use_Cohere_English_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "cohere.embed-english-v3")    
    dimension = 1024
else:
    lim_emb = None

llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f2d201fca90>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)

# 3. Load all Json files

In [5]:
from local_utils.proc_docs import get_load_json, show_doc_json

In [6]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/all_processed_data.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

.[]
all json files:  1
all items:  1732


# 4. Index 생성

## Index 이름 결정

In [7]:
index_name = "<Type index name>"
index_name = "v01-genai-poc-parent-doc-retriever"

## Index 스키마 정의

In [8]:
index_body = {
    'settings': {
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},
                               'family_tree' : {'type': 'keyword'},                                        
                               'parent_id' : {'type': 'keyword'},                    
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': f"{dimension}"  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [9]:
from local_utils.proc_docs import get_parameter

In [10]:
import boto3
region = boto3.Session().region_name
ssm = boto3.client('ssm', region)


opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_user_id',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_user_password',
)


In [11]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [12]:
from local_utils.opensearch import opensearch_utils

In [13]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [14]:
from local_utils.opensearch import opensearch_utils

In [15]:

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=v01-genai-poc-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v01-genai-poc-parent-doc-retriever'}
Index is created
{'v01-genai-poc-parent-doc-retriever': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'family_tree': {'type': 'keyword'},
                                                                                                'last_updated': {'type': 'date'},
                                                                                                'parent_id': {'type': 'keyword'},
                                                                                                'project': {'type': 'keyword'},
                                                                                                'seq_num': {'type': 'long'},
                                                                                                'source': {

## [중요] Parameter Store 에 index name 저장
- opensearch_index_name 변수 값을 추후 노트북에서 사용합니다.

In [16]:
from local_utils.proc_docs import put_parameter

put_parameter(
    boto3_clinet = ssm, 
    parameter_name = "opensearch_index_name",
    parameter_value = f'{index_name}'
)

Parameter stored successfully.
{'Version': 24, 'Tier': 'Standard', 'ResponseMetadata': {'RequestId': 'e4ff9544-b366-481b-88ee-fb2d545d1cc7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Fri, 29 Mar 2024 07:37:09 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '32', 'connection': 'keep-alive', 'x-amzn-requestid': 'e4ff9544-b366-481b-88ee-fb2d545d1cc7'}, 'RetryAttempts': 0}}


In [17]:
opensearch_index_name = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_index_name',
)

print ("opensearch_index_name: ", opensearch_index_name)



opensearch_index_name:  v01-genai-poc-parent-doc-retriever


## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [18]:
from langchain.vectorstores import OpenSearchVectorSearch

In [19]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

# 6. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [20]:
parent_chunk_size = 4096
parent_chunk_overlap = 0

child_chunk_size = 1024
child_chunk_overlap = 256

opensearch_parent_key_name = "parent_id"
opensearch_family_tree_key_name = "family_tree"


In [21]:
from local_utils.proc_docs import create_parent_chunk, create_child_chunk

## Parent Chunking

create_parent_chunk() 아래와 같은 작업을 합니다.
- all_docs 에 있는 문서를 parent_chunk_size 만큼으로 청킹 합니다.
- Parent Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: parent
    - parent_id : None

In [22]:
parent_chunk_docs = create_parent_chunk(all_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 2325


In [23]:
parent_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'parent', 'parent_id': None})]

### OpenSearch 에 parent chunk 삽입
- 아래는 약 6분 정도 걸립니다.

In [24]:
%%time

parent_ids = vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 6.45 s, sys: 362 ms, total: 6.81 s
Wall time: 5min 6s


In [25]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 2325, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


삽입된 Parent Chunk 의 첫번째를 확인 합니다. family_tree, parent_id 의 값을 확인 하세요.

In [26]:
def show_opensearch_doc_info(response):
    print("opensearch document id:" , response["_id"])
    print("family_tree:" , response["_source"]["metadata"]["family_tree"])
    print("parent document id:" , response["_source"]["metadata"]["parent_id"])
    print("parent document text: \n" , response["_source"]["text"])

response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: 7c043ce2-5b4c-40f9-a6bf-6714261c200f
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


## Child Chunking

### Child Chunk 생성

아래의 create_child_chunk() 는 다음과 같은 작업을 합니다.
- parent_chunk_docs 각각에 대해서 Child Chunk 를 생성 합니다. 
- Child Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: child
    - parent_id : parent 에 대한 OpenSearch document id

In [27]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 6884


### 생성된 Child 와 이에 대한 Parent 정보 확인

Child Chunk 한개에 대한 정보를 확인 합니다.

In [28]:
child_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'child', 'parent_id': '7c043ce2-5b4c-40f9-a6bf-6714261c200f'})]

Child 에 대한 Parent 정보 확인

In [29]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = index_name)
show_opensearch_doc_info(response)    


child's parent_id:  7c043ce2-5b4c-40f9-a6bf-6714261c200f

###### Search parent in OpenSearch
opensearch document id: 7c043ce2-5b4c-40f9-a6bf-6714261c200f
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


### OpenSearch 에 Child 삽입
- 아래는 약 13분 정도 소요 됩니다.

In [30]:
%%time

child_ids = vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  6884
CPU times: user 18.3 s, sys: 879 ms, total: 19.2 s
Wall time: 10min 40s


In [31]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 9209, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [32]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: a5d326e2-c9de-436e-9490-9f774f99885c
family_tree: child
parent document id: 7c043ce2-5b4c-40f9-a6bf-6714261c200f
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


# 7. 검색 테스트

In [33]:
from utils.rag import retriever_utils
from local_utils.rag import show_context_used
#from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### LLM 선택

In [34]:
from langchain_community.chat_models import BedrockChat

In [35]:
llm_text = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Haiku"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f2d201fca90>, model_id='anthropic.claude-3-haiku-20240307-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f2cfa095e40>])

### QA prompt and chain

In [36]:
from utils.rag import prompt_repo
from utils.rag import qa_chain
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [37]:
system_prompt = prompt_repo.get_system_prompt()
pprint (system_prompt)
system_prompt = """
                You are a master answer bot designed to answer user's questions.
                I'm going to give you contexts which consist of texts, tables and images.
                Read the contexts carefully, because I'm going to ask you a question about it.
                """
human_prompt = """
                Here is the contexts as texts: <contexts>{contexts}</contexts>

                First, find a few paragraphs or sentences from the contexts that are most relevant to answering the question.
                Then, answer the question as much as you can.

                Skip the preamble and go straight into the answer.
                Don't insert any XML tag such as <contexts> and </contexts> when answering.
                Answer in Korean.

                Here is the question: <question>{question}</question>

                If the question cannot be answered by the contexts, say "No relevant contexts".
               """
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)
prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm_text | StrOutputParser()

('\n'
 '                        You are a master answer bot designed to answer '
 "user's questions.\n"
 "                        I'm going to give you contexts which consist of "
 'texts, tables and images.\n'
 "                        Read the contexts carefully, because I'm going to "
 'ask you a question about it.\n'
 '                        ')


## Lexical 검색

In [38]:
#query = "중지된 경우 이체"
query = "vidio max size?"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [39]:
similar_docs_lexical = retriever_utils.get_lexical_similar_docs(
    index_name=index_name,
    os_client=os_client,
    query=query,
    k=5,
    filter=search_filter
)
show_context_used(similar_docs_lexical)

-----------------------------------------------
1. Chunk: 1008 Characters
-----------------------------------------------
. Invalid APK version name. Invalid APK version name. Please change the version name and try
uploading again. 4090001 File too large. Max 4000 devices allowed per upload. Try splitting the
device upload list to reduce the size of .csv you are uploading. Note that there is a max of 4000
devices allowed per upload. 4150000 File type is not supported. The file you are trying to upload is
not supported. Knox Configure only accepts .csv files. App name cannot contain special characters.
Rename the app so it does not contain any special or forbidden characters. App version is invalid.
An app you are trying to upload is invalid. Verify that the version you're attempting to upload is
correct. Try to re-upload the app again. CSV file is required. A system error occurred. Verify that
you've attached a .csv file, try to re-attach the .csv and upload it again. If the problem pe

In [40]:
answer = chain.invoke(
    {
        "contexts": similar_docs_lexical,
        "question": query
    }
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

이 문서들에는 "vidio max size"에 대한 정보가 포함되어 있지 않습니다. 문서들은 Knox Configure 사용자 인터페이스에서 발생할 수 있는 오류 메시지와 관련된 내용을 다루고 있습니다. 따라서 이 질문은 이 문맥에서는 답할 수 없습니다.##############################
query: 
 vidio max size?
answer: 
 이 문서들에는 "vidio max size"에 대한 정보가 포함되어 있지 않습니다. 문서들은 Knox Configure 사용자 인터페이스에서 발생할 수 있는 오류 메시지와 관련된 내용을 다루고 있습니다. 따라서 이 질문은 이 문맥에서는 답할 수 없습니다.


## 시맨틱 검색

In [41]:
query = "vidio max size?"
#query = "중지된 경우 이체"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [42]:
similar_docs_semantic = retriever_utils.get_semantic_similar_docs(
    index_name=index_name,
    os_client=os_client,
    llm_emb=llm_emb,
    query=query,
    k=5,
    boolean_filter=search_filter,
    hybrid=False
)
show_context_used(similar_docs_semantic)

-----------------------------------------------
1. Chunk: 457 Characters
-----------------------------------------------
. Rectangular viewfinder width Set the width of the rectangular viewfinder, specified in screen
width percentage. Only active if View finder selection is set to Rectangular. Available values are
0% to 100%. Default value is 90%. Rectangular viewfinder height Set the height of the rectangular
viewfinder specified in screen height percentage. Only active if View finder selection is set to
Rectangular. Available values are 0% to 100%. Default value is 40%.
metadata:
 {'source': 'all_processed_data.json', 'seq_num': 566, 'title': 'Configure scan engine settings',
'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-guides/configure-scan-engine-
settings', 'project': 'KCAP', 'last_updated': '2023-10-16', 'family_tree': 'child', 'parent_id':
'791663ba-568d-4b59-9619-76ff6ffaad85', 'id': 'ad9589d8-fd45-41aa-ad54-3c6f3088c70a'}
-------------------------------------

In [43]:
answer = chain.invoke(
    {
        "contexts": similar_docs_semantic,
        "question": query
    }
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

비디오 파일 크기는 다음과 같습니다:

Android Enterprise 정책에 따르면, 화면 보호기에 사용할 수 있는 비디오 파일의 최대 크기는 50 MB입니다. 파일은 MP4 또는 MKV 형식이어야 합니다.

따라서 이 문맥에 따르면 화면 보호기에 사용될 수 있는 비디오 파일의 최대 크기는 50 MB입니다.##############################
query: 
 vidio max size?
answer: 
 비디오 파일 크기는 다음과 같습니다:

Android Enterprise 정책에 따르면, 화면 보호기에 사용할 수 있는 비디오 파일의 최대 크기는 50 MB입니다. 파일은 MP4 또는 MKV 형식이어야 합니다.

따라서 이 문맥에 따르면 화면 보호기에 사용될 수 있는 비디오 파일의 최대 크기는 50 MB입니다.


# 완료 하셨습니다. 다음 노트북으로 가세요.

# [옵션]. 검증 인덱스 생성

아래 부분은 전체 문서의 일부(10%) 만을 샘플링하여 검증용으로 사용합니다. 
그래서 검증용 인덱스를 생성 합니다.

## Index 이름 결정

In [ ]:
eval_index_name = "<Type Index Name>"
eval_index_name = "v02-genai-poc-knox-eval-parent-doc-retriever"

## 인덱스 이름을 Parameter Store 에 저장

In [ ]:
from local_utils.proc_docs import put_parameter

put_parameter(
    boto3_clinet = ssm, 
    parameter_name = "opensearch_evaluation_index_name",
    parameter_value = f'{eval_index_name}'
)

In [ ]:
opensearch_index_name = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_evaluation_index_name',
)

print ("opensearch_index_name: ", opensearch_index_name)



## Sampling

In [ ]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.05, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [ ]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

## 검증 인덱스 연결 오브젝트 생성

In [ ]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
eval_vector_db

## Parent Chunking

In [ ]:
parent_chunk_docs = create_parent_chunk(eval_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, 
                                        parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


In [ ]:
parent_chunk_docs[0:1]

In [ ]:
%%time

parent_ids = eval_vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



In [ ]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


In [ ]:
response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = eval_index_name)
show_opensearch_doc_info(response)    

## Child Chunking

In [ ]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, 
                                      parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


In [ ]:
child_chunk_docs[0:1]

In [ ]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


In [ ]:
%%time

child_ids = eval_vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

In [ ]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


In [ ]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[5], index_name = eval_index_name)
show_opensearch_doc_info(response)    

In [ ]:
parent_id = response["_source"]["metadata"]["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


## 검색 테스트

In [ ]:
q = "'how to use barcode"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': f"{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

In [ ]:
response = opensearch_utils.search_document(os_client, query, eval_index_name)
opensearch_utils.parse_keyword_response(response, show_size=5)

In [ ]:
eval_vector_db.similarity_search(q, k=7)